# High-level MXNet Example

**In the interest of comparison; a common (custom) data-generator (called yield_mb(X, y, batchsize=64, shuffle=False)) was originally used for all other frameworks - but not for MXNet. I have reproduced the MXNet example using this same generator (wrapping the results in the mx.io.DataBatch class) to test if MXNet is faster than other frameworks just because I was using its own data-generator. This does not appear to be the case. **

In [1]:
import os
import sys
import numpy as np
import mxnet as mx
from common.params import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("MXNet: ", mx.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.14.1
MXNet:  0.12.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [4]:
def create_symbol(n_classes=N_CLASSES):
    data = mx.symbol.Variable('data')
    # size = [(old-size - kernel + 2*padding)/stride]+1
    # if kernel = 3, pad with 1 either side
    conv1 = mx.symbol.Convolution(data=data, num_filter=50, pad=(1,1), kernel=(3,3))
    relu1 = mx.symbol.Activation(data=conv1, act_type="relu")
    conv2 = mx.symbol.Convolution(data=relu1, num_filter=50, pad=(1,1), kernel=(3,3))
    pool1 = mx.symbol.Pooling(data=conv2, pool_type="max", kernel=(2,2), stride=(2,2))
    relu2 = mx.symbol.Activation(data=pool1, act_type="relu")
    drop1 = mx.symbol.Dropout(data=relu2, p=0.25)
    
    conv3 = mx.symbol.Convolution(data=drop1, num_filter=100, pad=(1,1), kernel=(3,3))
    relu3 = mx.symbol.Activation(data=conv3, act_type="relu")
    conv4 = mx.symbol.Convolution(data=relu3, num_filter=100, pad=(1,1), kernel=(3,3))
    pool2 = mx.symbol.Pooling(data=conv4, pool_type="max", kernel=(2,2), stride=(2,2))
    relu4 = mx.symbol.Activation(data=pool2, act_type="relu")
    drop2 = mx.symbol.Dropout(data=relu4, p=0.25)
           
    flat1 = mx.symbol.Flatten(data=drop2)
    fc1 = mx.symbol.FullyConnected(data=flat1, num_hidden=512)
    relu7 = mx.symbol.Activation(data=fc1, act_type="relu")
    drop4 = mx.symbol.Dropout(data=relu7, p=0.5)
    fc2 = mx.symbol.FullyConnected(data=drop4, num_hidden=n_classes) 
    
    input_y = mx.symbol.Variable('softmax_label')  
    m = mx.symbol.SoftmaxOutput(data=fc2, label=input_y, name="softmax")
    return m

In [5]:
def init_model(m, batchsize=BATCHSIZE, lr=LR, momentum=MOMENTUM):
    ctx = [mx.gpu(0)]
    mod = mx.mod.Module(context=ctx, symbol=m)
    mod.bind(data_shapes=[('data', (batchsize, 3, 32, 32))],
             label_shapes=[('softmax_label', (batchsize,))])

    # Glorot-uniform initializer
    mod.init_params(initializer=mx.init.Xavier(rnd_type='uniform'))
    mod.init_optimizer(optimizer='sgd', 
                       optimizer_params=(('learning_rate', lr), ('momentum', momentum), ))
    return mod

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)

# Load data-iterator
#train_iter = mx.io.NDArrayIter(x_train, y_train, BATCHSIZE, shuffle=True)
# Use custom iterator instead of mx.io.NDArrayIter() for consistency
# Wrap as DataBatch class
wrapper_db = lambda args: mx.io.DataBatch(data=[mx.nd.array(args[0])], label=[mx.nd.array(args[1])])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 655 ms, sys: 581 ms, total: 1.24 s
Wall time: 1.23 s


In [7]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 3.56 ms, sys: 0 ns, total: 3.56 ms
Wall time: 2.91 ms


In [8]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 933 ms, sys: 976 ms, total: 1.91 s
Wall time: 2.42 s


In [9]:
%%time
# Main training loop: 51s
metric = mx.metric.create('acc')
for j in range(EPOCHS):
    metric.reset()
    for batch in map(wrapper_db, yield_mb(x_train, y_train, BATCHSIZE, shuffle=True)):
        model.forward(batch, is_train=True) 
        model.update_metric(metric, batch.label)
        model.backward()              
        model.update()
    print('Epoch %d, Training %s' % (j, metric.get()))

Epoch 0, Training ('accuracy', 0.32702464788732394)
Epoch 1, Training ('accuracy', 0.48269446222791296)
Epoch 2, Training ('accuracy', 0.5619998399487837)
Epoch 3, Training ('accuracy', 0.6212788092189501)
Epoch 4, Training ('accuracy', 0.6703545134443022)
Epoch 5, Training ('accuracy', 0.7117277528809219)
Epoch 6, Training ('accuracy', 0.7358554737516005)
Epoch 7, Training ('accuracy', 0.7603032970550576)
Epoch 8, Training ('accuracy', 0.7789492637644047)
Epoch 9, Training ('accuracy', 0.796354833546735)
CPU times: user 48.1 s, sys: 18.1 s, total: 1min 6s
Wall time: 51.7 s


In [10]:
%%time
# Main evaluation loop: 311ms
y_guess = model.predict(mx.io.NDArrayIter(x_test, batch_size=BATCHSIZE, shuffle=False))
y_guess = np.argmax(y_guess.asnumpy(), axis=-1)

CPU times: user 361 ms, sys: 146 ms, total: 508 ms
Wall time: 310 ms


In [11]:
print("Accuracy: ", 1.*sum(y_guess == y_test)/len(y_guess))

Accuracy:  0.7739
